### Proj Asn 01 : Colin Kirby

In [1]:
# Imports.
import pygame
import moderngl
import glm
import numpy as np
from LoadObject import getObjectData

pygame 2.6.0 (SDL 2.28.4, Python 3.12.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Initialize Window for Display.
def init_window(width=800, height=600):
    pygame.init()
    pygame.display.set_mode((width, height), pygame.OPENGL | pygame.DOUBLEBUF)
    pygame.display.set_caption("Project Assignment 01: Colin Kirby")

    ctx = moderngl.create_context()
    ctx.enable(moderngl.CULL_FACE)
    ctx.enable(moderngl.DEPTH_TEST)

    return ctx

ctx = init_window()


In [3]:
# Import and Load 3D Models.
vertex_data_platform, _ = getObjectData("cube.obj", normal=True, texture=True)
vertex_data_teapot, _ = getObjectData("teapot_with_texCoords.obj", normal=True, texture=True)
vertex_data_light, _ = getObjectData("20_icosahedron.obj")

Normal exists
texture exists
Normal exists
texture exists


In [4]:
# Initialize VAOs and Shader Programs.
vao_platform = None
vao_teapot = None
vao_light_source = None
program_platform = None
program_teapot = None
program_light_source = None

In [5]:
# Start w/ Point Light.
is_point_light = True

In [6]:
# Calculate 3D Position of Light Source.
def get_light_position(angle):
    distance = 15.0  
    height_angle = glm.radians(45.0)  

    x = distance * glm.cos(angle) * glm.sin(height_angle)
    y = distance * glm.cos(height_angle)
    z = distance * glm.sin(angle) * glm.sin(height_angle)

    return glm.vec3(x, y, z)

In [7]:
# Return Normalized Light Vector.
def get_light_direction(light_position):
    return glm.normalize(-light_position)

In [8]:
# Create Scaling Matrix.
light_scale_matrix = glm.scale(glm.mat4(1.0), glm.vec3(0.1))

In [9]:
# Light Source Shader Program.
program_light_source = ctx.program(
    vertex_shader="""
    #version 330
    in vec3 in_position;
    uniform mat4 model;
    uniform mat4 view;
    uniform mat4 proj;

    void main() {
        gl_Position = proj * view * model * vec4(in_position, 1.0);
    }
    """,
    fragment_shader="""
    #version 330
    out vec4 frag_color;
    void main() {
        frag_color = vec4(1.0);  // White color for the light source object
    }
    """
)

vertex_data_light_np = np.array(vertex_data_light, dtype='f4')
vbo_light_source = ctx.buffer(vertex_data_light_np.tobytes())
vao_light_source = ctx.vertex_array(
    program_light_source,
    [(vbo_light_source, '3f', 'in_position')],
)

In [10]:
# Set Angle Of Light to 0.
light_angle = 0.0

In [11]:
# Load Image Func.
def load_image(image_file, channels="RGBA", flip_x=False, flip_y=False):
    texture_img = pygame.image.load(image_file)
    if flip_x or flip_y:
        texture_img = pygame.transform.flip(texture_img, flip_x, flip_y)
    texture_data = pygame.image.tobytes(texture_img, channels, True)
    return texture_data, texture_img.get_size()

In [12]:
grass_texture_data, grass_texture_size = load_image("grass.jpg", channels="RGB", flip_y=True)
grass_texture = ctx.texture(grass_texture_size, 3, grass_texture_data)
grass_texture.build_mipmaps()

gold_texture_data, gold_texture_size = load_image("gold.jpg", channels="RGB", flip_y=True)
gold_texture = ctx.texture(gold_texture_size, 3, gold_texture_data)
gold_texture.build_mipmaps()

In [13]:
# Camera and Projection Matrices.
fov = 60.0
width, height = 800, 600
aspect_ratio = width / height
near_plane = 10.0
far_plane = 45.0

In [14]:
# Set Proj and View Matrix.
proj_matrix = glm.perspective(glm.radians(fov), aspect_ratio, near_plane, far_plane)
view_matrix = glm.lookAt(
    glm.vec3(0.0, 10.0, 20.0),  # Camera Position
    glm.vec3(0.0, 0.0, 0.0),    # Look at point
    glm.vec3(0.0, 1.0, 0.0)     # Up vector
)

In [15]:
# Vertex Data for Platform.
def set_vert_data_platform():
    global vao_platform, program_platform
    vertex_data_platform_np = np.array(vertex_data_platform, dtype='f4')
    vbo_platform = ctx.buffer(vertex_data_platform_np.tobytes())

    program_platform = ctx.program(
        vertex_shader="""
        #version 330
        layout(location = 0) in vec3 in_position;
        layout(location = 1) in vec3 in_normal;
        layout(location = 2) in vec2 in_uv;
        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 proj;
        out vec3 frag_normal;
        out vec3 frag_position;
        out vec2 frag_uv;

        void main() {
            mat3 normal_matrix = transpose(inverse(mat3(model)));
            frag_normal = normalize(normal_matrix * in_normal);
            frag_position = vec3(model * vec4(in_position, 1.0));
            frag_uv = in_uv;
            gl_Position = proj * view * vec4(frag_position, 1.0);
        }
        """,
        fragment_shader="""
        #version 330
        in vec3 frag_normal;
        in vec3 frag_position;
        in vec2 frag_uv;
        out vec4 frag_color;
        uniform vec3 light_position;
        uniform vec3 light_direction;
        uniform bool is_point_light;
        uniform sampler2D material_texture;

        void main() {
            vec3 material_color = texture(material_texture, frag_uv).rgb;
            vec3 normal = normalize(frag_normal);
            vec3 light_dir;
            if (is_point_light) {
                light_dir = normalize(light_position - frag_position);
            } else {
                light_dir = normalize(-light_direction);
            }
            float diff = max(dot(normal, light_dir), 0.0);
            vec3 diffuse = diff * material_color;
            frag_color = vec4(diffuse, 1.0);
        }
        """
    )

    vao_platform = ctx.vertex_array(
        program_platform,
        [(vbo_platform, '3f 3f 2f', 'in_position', 'in_normal', 'in_uv')]
    )

In [16]:
# Vertex Data for Teapot.
def set_vert_data_teapot():
    global vao_teapot, program_teapot, vbo_teapot
    vertex_data_teapot_np = np.array(vertex_data_teapot, dtype='f4')
    vbo_teapot = ctx.buffer(vertex_data_teapot_np.tobytes())

    program_teapot = ctx.program(
        vertex_shader="""
        #version 330
        layout(location = 0) in vec3 in_position;
        layout(location = 1) in vec3 in_normal;
        layout(location = 2) in vec2 in_uv;
        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 proj;
        out vec3 frag_normal;
        out vec3 frag_position;
        out vec2 frag_uv;

        void main() {
            mat3 normal_matrix = transpose(inverse(mat3(model)));
            frag_normal = normalize(normal_matrix * in_normal);
            frag_position = vec3(model * vec4(in_position, 1.0));
            frag_uv = in_uv;
            gl_Position = proj * view * vec4(frag_position, 1.0);
        }
        """,
        fragment_shader="""
        #version 330
        in vec3 frag_normal;
        in vec3 frag_position;
        in vec2 frag_uv;
        out vec4 color;
        uniform vec3 light_position;
        uniform vec3 light_direction;
        uniform bool is_point_light;
        uniform sampler2D material_texture;
        uniform bool metal;
        uniform vec3 eyePosition;
        const float shininess = 25.0;

        void main() {
            vec3 material_color = texture(material_texture, frag_uv).rgb;
            vec3 normal = normalize(frag_normal);
            vec3 light_dir;
            if (is_point_light) {
                light_dir = normalize(light_position - frag_position);
            } else {
                light_dir = normalize(-light_direction);
            }
            if (metal) {
                vec3 view_dir = normalize(eyePosition - frag_position);
                vec3 halfway_dir = normalize(light_dir + view_dir);
                float spec = pow(max(dot(normal, halfway_dir), 0.0), shininess);
                vec3 specular = spec * material_color;
                color = vec4(specular, 1.0);
            } else {
                float diff = max(dot(normal, light_dir), 0.0);
                vec3 diffuse = diff * material_color;
                color = vec4(diffuse, 1.0);
            }
        }
        """
    )

    vao_teapot = ctx.vertex_array(
        program_teapot,
        [(vbo_teapot, '3f 3f 2f', 'in_position', 'in_normal', 'in_uv')]
    )

program_shadow = ctx.program(
    vertex_shader="""
    #version 330
    layout(location = 0) in vec3 in_position;

    uniform mat4 model;
    uniform mat4 view;
    uniform mat4 proj;
    uniform vec4 light_position;
    uniform vec3 plane_normal;
    uniform vec3 plane_point;
    uniform float shadow_scale;

    void main() {
        vec4 world_position = model * vec4(in_position, 1.0);

        vec3 L;
        if (light_position.w == 0.0) {
            // Directional light
            L = normalize(light_position.xyz);
        } else {
            // Point light
            L = normalize(light_position.xyz - world_position.xyz);
        }

        float d = dot(plane_normal, (plane_point - world_position.xyz)) / dot(plane_normal, L);

        vec3 shadow_position = world_position.xyz + d * L;

        // Scale the shadow
        vec3 offset = shadow_position - plane_point;
        offset *= shadow_scale;
        shadow_position = plane_point + offset;

        gl_Position = proj * view * vec4(shadow_position, 1.0);
    }
    """,
    fragment_shader="""
    #version 330
    out vec4 frag_color;
    void main() {
        frag_color = vec4(0.0, 0.0, 0.0, 0.5);  // Semi-transparent black
    }
    """
)

In [17]:
# Sets the Vertex Data by Referencing Funcs.
set_vert_data_platform()
set_vert_data_teapot()

In [18]:
# Load Teapot w/o Normals or Textures.
vertex_data_teapot_shadow, _ = getObjectData("teapot_with_texCoords.obj", normal=False, texture=False)
vertex_data_teapot_shadow_np = np.array(vertex_data_teapot_shadow, dtype='f4')
vbo_teapot_shadow = ctx.buffer(vertex_data_teapot_shadow_np.tobytes())

In [19]:
# Create VAO for Rendering Shadow.
vao_shadow = ctx.vertex_array(
    program_shadow,
    [(vbo_teapot_shadow, '3f', 'in_position')],
)

In [ ]:
# Main Rendering Loop.
running = True
while running:
    # Event Handler.
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_l:
                is_point_light = not is_point_light

    # Update Light Position.
    light_angle += 0.0002
    light_position = get_light_position(light_angle)
    light_direction = get_light_direction(light_position)

    # Transformation Matrix for Platform.
    model_matrix_platform = glm.scale(glm.mat4(1.0), glm.vec3(10.0, 0.05, 10.0))
    model_matrix_platform = glm.translate(model_matrix_platform, glm.vec3(0.0, 0.1, 0.0))

    # Transformation Matrix for Teapot.
    model_matrix_teapot = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 1.5, 0.0))
    model_matrix_teapot = glm.scale(model_matrix_teapot, glm.vec3(0.29))
    
    # Calcualte Eye Position.
    eye_position = glm.vec3(glm.inverse(view_matrix)[3])

    # Clear Screen.
    ctx.clear(0.1, 0.1, 0.1)
    ctx.enable(moderngl.DEPTH_TEST)

    # Convert View and Project Matrixes for Shader Unifrom.
    view_bytes = np.array(view_matrix, dtype='f4').T.tobytes()
    proj_bytes = np.array(proj_matrix, dtype='f4').T.tobytes()

    # Loading Platform w/ Textures.
    grass_texture.use(0)
    program_platform['model'].write(np.array(model_matrix_platform, dtype='f4').T.tobytes())
    program_platform['view'].write(view_bytes)
    program_platform['proj'].write(proj_bytes)
    program_platform['light_position'].value = tuple(light_position)
    program_platform['light_direction'].value = tuple(light_direction)
    program_platform['is_point_light'].value = is_point_light
    program_platform['material_texture'].value = 0
    vao_platform.render(moderngl.TRIANGLES)

    # Loading Teapot w/ Textures.
    gold_texture.use(0)
    program_teapot['model'].write(np.array(model_matrix_teapot, dtype='f4').T.tobytes())
    program_teapot['view'].write(view_bytes)
    program_teapot['proj'].write(proj_bytes)
    program_teapot['light_position'].value = tuple(light_position)
    program_teapot['light_direction'].value = tuple(light_direction)
    program_teapot['is_point_light'].value = is_point_light
    program_teapot['material_texture'].value = 0
    program_teapot['metal'].value = True
    program_teapot['eyePosition'].value = tuple(eye_position)
    vao_teapot.render(moderngl.TRIANGLES)

    # Load Light Source.
    model_light_matrix = glm.translate(glm.mat4(1.0), light_position) * light_scale_matrix
    program_light_source['model'].write(np.array(model_light_matrix, dtype='f4').T.tobytes())
    program_light_source['view'].write(view_bytes)
    program_light_source['proj'].write(proj_bytes)
    vao_light_source.render(moderngl.TRIANGLES)

    # Load Shadow.
    ctx.disable(moderngl.CULL_FACE)

    # Set Shader Uniforms.
    program_shadow['model'].write(np.array(model_matrix_teapot, dtype='f4').T.tobytes())
    program_shadow['view'].write(view_bytes)
    program_shadow['proj'].write(proj_bytes)
    program_shadow['light_position'].value = tuple(light_position) + (0.0 if not is_point_light else 1.0,)
    program_shadow['plane_normal'].value = (0.0, 1.0, 0.0)
    program_shadow['plane_point'].value = (0.0, 0.1, 0.0)  # Platform at y = 0.1

    shadow_scale_value = 1.0
    program_shadow['shadow_scale'].value = shadow_scale_value

    vao_shadow.render(moderngl.TRIANGLES)

    ctx.enable(moderngl.CULL_FACE)

    pygame.display.flip()

# Clean Up.
pygame.quit()
